In [ ]:

from zenml import pipeline, step
from sklearn.datasets import load_digits
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from typing import Any, Tuple

@step
def load_data() -> pd.DataFrame:
    data = load_digits()
    df = pd.DataFrame(data.data)
    df['target'] = data.target
    return df

@step
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the input DataFrame by dropping columns and filling missing values.
    """
    if 'unnecessary_column' in df.columns:
        df = df.drop(columns=['unnecessary_column'])

    df = df.fillna(df.median(numeric_only=True))

    df = df.fillna("no review")
    
    return df

@step
def train_model(df: pd.DataFrame) -> Tuple[Any, pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Trains a Linear Regression model on the cleaned DataFrame.
    Splits the data into train and test sets.
    """
    X = df.drop(columns=["target"])
    y = df["target"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    return model, X_train, X_test, y_train, y_test

@step
def evaluate_model(model: Any, X_test: pd.DataFrame, y_test: pd.Series) -> float:
    """
    Evaluates the model using Mean Squared Error (MSE).
    """
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error: {mse}")
    return mse

@pipeline
def training_pipeline():
    df = load_data()
    cleaned_df = clean_data(df)
    model, X_train, X_test, y_train, y_test = train_model(cleaned_df)
    evaluate_model(model, X_test, y_test)

if __name__ == "__main__":
    training_pipeline()


Initiating a new run for the pipeline: training_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step load_data.
[evaluate_model] Step clean_data has started.
Step clean_data has finished in 1.537s.
Step train_model has started.
Step train_model has finished in 4.203s.
Step evaluate_model has started.
[evaluate_model] Mean Squared Error: 3.2599689226111743
Step evaluate_model has finished in 2.410s.
Pipeline run has finished in 8.633s.
